In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
%%capture
!pip install wandb

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4095 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Phi-3` format for conversation style finetunes. We use [Open Assistant conversations](https://huggingface.co/datasets/philschmid/guanaco-sharegpt-style) in ShareGPT style. Phi-3 renders multi turn conversations like below:

```
<|user|>
Hi!<|end|>
<|assistant|>
Hello! How are you?<|end|>
<|user|>
I'm doing great! And you?<|end|>

```

**[NOTE]** To train only on completions (ignoring the user's input) read Unsloth's docs [here](https://github.com/unslothai/unsloth/wiki#train-on-completions--responses-only-do-not-train-on-inputs).

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old` and our own optimized `unsloth` template.

Note ShareGPT uses `{"from": "human", "value" : "Hi"}` and not `{"role": "user", "content" : "Hi"}`, so we use `mapping` to map it.

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [ ]:
from google.colab import userdata
import os

os.environ["HF_HUB_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')

In [ ]:
from huggingface_hub import login
import os

login(token=os.getenv("HF_HUB_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset, Features, Value, DatasetDict, concatenate_datasets

features = Features({
    'input': Value('string'),
    'output': Value('string'),
    'instruct': Value('string'),
    'dataset_type': Value('string'),
    'dataloader_name': Value('string')
})

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
)

def formatting_prompts_func(row):
    convos = [
        {
        "content": f"{row['instruct']}\n Input: {row['input']} Output:",
        "role": "user"
    }, {
        "content": f"{row['output']}",
        "role": "assistant"
    }]
    texts = tokenizer.apply_chat_template(convos, tokenize = False, add_generation_prompt = False)
    return { "text" : texts, }

In [ ]:
from collections import Counter

def custom_sample(dataset, sampling_fractions):
    sampled_datasets = []
    for name, fraction in sampling_fractions.items():
        filtered_dataset = dataset.filter(lambda x: x['dataloader_name'] == name)
        if fraction < 1.0:
            sampled_dataset = DatasetDict({split: filtered_dataset[split].train_test_split(train_size=fraction, seed=42)['train'] for split in filtered_dataset})
        else:
            sampled_dataset = filtered_dataset
        sampled_datasets.append(sampled_dataset)

    return DatasetDict({split: concatenate_datasets([d[split] for d in sampled_datasets]) for split in filtered_dataset})


def value_counts(data):
  value_counts = Counter(data)
  print(value_counts)

In [ ]:
artificial_ds = load_dataset("ostapbodnar/ua-gec-pos-ner-artificial", data_dir="small", features=features)

value_counts(artificial_ds["train"]['dataloader_name'])

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

train.tsv:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

valid.tsv:   0%|          | 0.00/311M [00:00<?, ?B/s]

test.tsv:   0%|          | 0.00/462M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Counter({'PapersDataset': 61100, 'UbertextV2Dataset': 34708})


In [ ]:
sampling_fractions = {
    "PapersDataset": 0.65,
    "UbertextV2Dataset": 1.0,
}


artificial_ds = artificial_ds.filter(lambda x: x['dataloader_name'] != "null")
artificial_ds = custom_sample(artificial_ds, sampling_fractions)
artificial_ds = artificial_ds.shuffle(seed=42)

artificial_ds = DatasetDict({
    "train": artificial_ds["train"].shuffle(seed=42).select(range(10000)),
    "test": artificial_ds["test"].shuffle(seed=42).select(range(500)),
})

value_counts(artificial_ds["train"]['dataloader_name'])

Filter:   0%|          | 0/95808 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25549 [00:00<?, ? examples/s]

Filter:   0%|          | 0/38323 [00:00<?, ? examples/s]

Filter:   0%|          | 0/95808 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25549 [00:00<?, ? examples/s]

Filter:   0%|          | 0/38323 [00:00<?, ? examples/s]

Filter:   0%|          | 0/95808 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25549 [00:00<?, ? examples/s]

Filter:   0%|          | 0/38323 [00:00<?, ? examples/s]

Counter({'PapersDataset': 5347, 'UbertextV2Dataset': 4653})


In [ ]:
golden_ds = load_dataset("ostapbodnar/ua-gec-pos-ner-golden", features=features)

value_counts(golden_ds["train"]['dataloader_name'])

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

train.tsv:   0%|          | 0.00/597M [00:00<?, ?B/s]

valid.tsv:   0%|          | 0.00/149M [00:00<?, ?B/s]

test.tsv:   0%|          | 0.00/209M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Counter({'NewsKeywordDataset': 96350, 'NewsTopicClassificationDataset': 96317, 'UaSqaudDataset': 8869, 'UaGecDataset': 4937, 'MovaInstPosDataset': 4627, 'ZnoDataset': 2443, 'NerDataset': 224, 'WscDataset': 177, None: 16})


In [ ]:

sampling_fractions = {
    "UaGecDataset": 0.95,
    "UaSqaudDataset": 0.95,
    "NewsTopicClassificationDataset": 0.03,
    "NewsKeywordDataset": 0.03,
    "WscDataset": 1.0,
    "NerDataset": 1.0,
    'MovaInstPosDataset': 0.95,
    'ZnoDataset': 0.65,
}


golden_ds = golden_ds.filter(lambda x: x['dataloader_name'] != "null")
golden_ds = custom_sample(golden_ds, sampling_fractions)
golden_ds = golden_ds.shuffle(seed=42)

golden_ds = DatasetDict({
    "train": golden_ds["train"].shuffle(seed=42).select(range(10000)),
    "test": golden_ds["test"].shuffle(seed=42).select(range(500)),
})

value_counts(golden_ds["train"]['dataloader_name'])

Filter:   0%|          | 0/213960 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53490 [00:00<?, ? examples/s]

Filter:   0%|          | 0/78990 [00:00<?, ? examples/s]

Filter:   0%|          | 0/213960 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53490 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/78990 [00:00<?, ? examples/s]

Filter:   0%|          | 0/213960 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53490 [00:00<?, ? examples/s]

Filter:   0%|          | 0/78990 [00:00<?, ? examples/s]

Counter({'UaSqaudDataset': 4366, 'UaGecDataset': 2428, 'NewsTopicClassificationDataset': 1527, 'NewsKeywordDataset': 1471, 'NerDataset': 115, 'WscDataset': 93})


In [ ]:
mixed_ds = DatasetDict({
    "train": concatenate_datasets([golden_ds["train"], artificial_ds["train"]]).shuffle(seed=42),
    "test": concatenate_datasets([golden_ds["test"], artificial_ds["test"]]).shuffle(seed=42),
})

dataset = mixed_ds
dataset = dataset.map(formatting_prompts_func, batched = False,)


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Let's see how the `Phi-3` format works by printing the 5th element

In [ ]:
value_counts(mixed_ds["train"]['dataloader_name'])

Counter({'PapersDataset': 5347, 'UbertextV2Dataset': 4653, 'UaSqaudDataset': 4366, 'UaGecDataset': 2428, 'NewsTopicClassificationDataset': 1527, 'NewsKeywordDataset': 1471, 'NerDataset': 115, 'WscDataset': 93})


In [ ]:
dataset['train'][5]

{'input': "«ідеальний розчин» — це ідеальна модель створена подумки, коли не беруть до уваги саме ці конкретні ознаки — об'єм і температуру, щоб віднайти загальніші, суттєвіші Ознаки, що притаманні усім розчинам. Це здійснюється здійснюється в інтересах пізнання; Ідеальне як об'єктивний фенмен, незалежний від людини, її свідомості («абсолютний розум», «абсолютна іыдея» у Гегеля, «вічні ідеї» у Платона тощо). Тепер зупинимося на з'ясуванні особливостей матеріального як реального, Об'єктивного, котре не залежить від людини та її свідомості. Для позначення того, що є об'єктивним, незалежним від свідомості людини у філософії вироблено уявлення про матерію. поняття матерії. Поняття «матерія», його Формування має свою історію. У ньайдавніших філософів античного світу вже була думка про першооснову всього існуючого. Нею, як відомо, вони вважали воду, повітря, вогонь- атом тощо. Міркування про це ми знаходимо, зокрема, у філософії Фалеса, Анаксімена, Геракліта, демокріта. Однак таке уявлення п

In [ ]:
print(dataset['train'][5]["text"])

<|user|>
Використай наведені нижче інструкції для обробки тексту.

    1. Виправлення граматичних помилок:
       - Формат: `<g ed="edit" et="тип помилки">error</g>`
       - Опис тегів:
         - Tag: категорія помилки, наприклад, G/Case, G/Tense, F/Style тощо.
         - error: текст до виправлення.
         - edit: текст після виправлення.
       - Типи помилок:
         - Орфографічні помилки:
           - Spelling: орфографічні помилки.
           - Punctuation: пунктуаційні помилки.
         - Граматичні помилки:
           - G/Case: неправильне використання відмінка.
           - G/Gender: неправильне використання роду.
           - G/Number: неправильне використання числа.
           - G/Aspect: неправильне використання аспекта дієслова.
           - G/Tense: неправильне використання часу дієслова.
           - G/VerbVoice: неправильне використання форми дієслова.
           - G/PartVoice: неправильне використання форми дієприкметника.
           - G/VerbAForm: неправильне вик

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruct', 'dataset_type', 'dataloader_name', 'text'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['input', 'output', 'instruct', 'dataset_type', 'dataloader_name', 'text'],
        num_rows: 1000
    })
})

If you're looking to make your own chat template, that also is possible! You must use the Jinja templating regime. We provide our own stripped down version of the `Unsloth template` which we find to be more efficient, and leverages ChatML, Zephyr and Alpaca styles.

More info on chat templates on [our wiki page!](https://github.com/unslothai/unsloth/wiki#chat-templates)

In [ ]:
unsloth_template = \
    "{{ bos_token }}"\
    "{{ 'You are a helpful assistant to the user\n' }}"\
    "{% for message in messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: ' }}"\
    "{% endif %}"
unsloth_eos_token = "eos_token"


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
# WANDB configuration (optional)

import wandb
import os

os.environ["PROJECT"]="phi3.5-mini-ua-mixed"

project_name = os.environ["PROJECT"]

wandb.init(project=project_name, name = project_name)

wandb: Currently logged in as: ostapbodnar (ostap-bodnar). Use `wandb login --relogin` to force relogin


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


new_model_id="ostapbodnar/Phi3.5-mini-instruct-UA-lora-unsloth-mixed"

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset['test'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 12,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 12,
        gradient_accumulation_steps = 6,
        per_device_eval_batch_size=4,
        warmup_steps = 5,
        num_train_epochs=3,
        learning_rate = 2e-4,
        push_to_hub=True,
        hub_model_id=new_model_id,
        hub_strategy="every_save",
        save_strategy="steps",
        save_steps=100,
        do_eval=True,
        evaluation_strategy="steps",
        eval_steps=100,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to="wandb",
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=12):   0%|          | 0/20000 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
12.418 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 12 | Gradient Accumulation steps = 6
\        /    Total batch size = 72 | Total steps = 831
 "-____-"     Number of trainable parameters = 29,884,416


Step,Training Loss,Validation Loss
100,0.543900,0.606476
200,0.477500,0.560280
300,0.435900,0.546315
400,0.467700,0.533601
500,0.454200,0.525674
600,0.421400,0.520727
700,0.407400,0.516628
800,0.396300,0.514510


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! Since we're using `Phi-3`, use `apply_chat_template` with `add_generation_prompt` set to `True` for inference.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "Продовжи послідовність фібоначі: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

## Upload 16bit version

In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [4]:
from google.colab import userdata
import os

os.environ["HF_HUB_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')

In [5]:
from huggingface_hub import login
import os

login(token=os.getenv("HF_HUB_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [9]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4095 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "ostapbodnar/Phi3.5-mini-instruct-UA-lora-unsloth-mixed",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32064, 3072)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
       

In [16]:
import torch

tokenized_input = tokenizer.apply_chat_template(
            [
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Давай відповіді на запитання. \n Input: Яка столиця України? Output:"},
    ],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")

with torch.no_grad():
    output = model.generate(
        input_ids=tokenized_input,
        # max_tokens=2096,
        # stop=["\nUser:", "<|endoftext|>", "</s>"],
    )

full_response = tokenizer.decode(output[0], skip_special_tokens=True)
full_response

'You are a helpful assistant Давай відповіді на запитання. \n Input: Яка столиця України? Output: Київ'

In [18]:
model.push_to_hub_merged("ostapbodnar/Phi3.5-mini-instruct-UA-lora-unsloth-mixed-16bit", tokenizer, save_method = "merged_16bit")

Unsloth: You are pushing to hub, but you passed your HF username = ostapbodnar.
We shall truncate ostapbodnar/Phi3.5-mini-instruct-UA-lora-unsloth-mixed-16bit to Phi3.5-mini-instruct-UA-lora-unsloth-mixed-16bit


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 55.22 out of 83.48 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 98.69it/s]


Unsloth: Saving tokenizer...

  0%|          | 0/1 [00:00<?, ?it/s]

...

README.md:   0%|          | 0.00/600 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/ostapbodnar/Phi3.5-mini-instruct-UA-lora-unsloth-mixed-16bit
